In [1]:
import pandas as pd
from openai import OpenAI
from sklearn.metrics import matthews_corrcoef
from tqdm import tqdm
import torch
from transformers import pipeline, AutoTokenizer
from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support, accuracy_score
from sklearn.metrics import f1_score as f1
from sklearn.metrics import matthews_corrcoef as mcc

/Users/mb7336/miniforge3/envs/sandbox/lib/python3.11/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/mb7336/miniforge3/envs/sandbox/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <EB3FF92A-5EB1-3EE8-AF8B-5923C1265422> /Users/mb7336/miniforge3/envs/sandbox/lib/python3.11/site-packages/torchvision/image.so
  Reason: tried: '/Users/mb7336/miniforge3/envs/sandbox/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/mb7336/miniforge3/envs/sandbox/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/mb7336/miniforge3/envs/sandbox/lib/python3.11/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/mb7336/miniforge3/envs/sandbox/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, 

In [10]:
motn = pd.read_csv('../data/freedom_test.csv', encoding = "ISO-8859-1")

motn = motn[~motn['text'].isna()]
motn.reset_index(drop = True, inplace = True)

# Llama 3 8B

In [4]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipe = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="mps",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

No Logit Bias

# Freedom and Rights

In [24]:
user_message = """You are a classifier that determines if a survey answer is about freedom and rights. The survey answer I will show is a response to the question "What does democracy mean to you?" 

Answers about freedom and rights reference "freedom" or "liberty" in the abstract, or reference a particular freedom such as the freedom to express one's opinion, freedom of religion, or other rights mentioned in the Bill of Rights. They may also contain references to freedom from government, freedom of speech, the Second Amendment, references to civil rights or substantive rights, and references to capitalism or free enterprise. 
However, the answer is not about freedom and rights if the answer is about voting rights the "right" to vote, or anything having to do with voting.
I will show you an answer to the question. Read the answer and then determine if the answer is about freedom and rights or not. Here is the respondent's answer:
{}
If the answer is about freedom and rights, return 1. If it is not about freedom and rights, return 0. Do not explain your answer, and only return the number.
"""
# change prompt to only voting != freedom and rights

In [25]:
%%time
data = freedom
res = []
for doc in data['text']:
    messages = [
        #{
            #"role": "system",
            #"content": system_message,
        #},
        {"role": "user", "content": user_message.format(doc)},
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=2, do_sample=False, return_full_text = False, pad_token_id=pipe.tokenizer.eos_token_id, temperature = 0)
    res.extend(outputs)

res = [text['generated_text'] for text in res]

# return a list of unique responses from the model
print(set(res))

# add results to the dataframe
#motn['llama'] = [1 if '1' in text else 0 for text in res]
# export results
#motn.to_csv('./motn_test_llama3.csv', index = False)

C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\transformers\generation\configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


{'1.', '0', '1'}
CPU times: total: 24min 31s
Wall time: 25min 57s


In [26]:
res = [num[0] for num in res]

In [33]:
freedom['llama'] = [1 if '1' in text else 0 for text in res]

C:\Users\mikeb\AppData\Local\Temp\ipykernel_24916\1176831904.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  freedom['llama'] = [1 if '1' in text else 0 for text in res]


In [13]:
accuracy_score(freedom['freedom_and_rights'], freedom['llama'])

0.8837671884459051

In [15]:
balanced_accuracy_score(freedom['freedom_and_rights'], freedom['llama'])

0.8723670689479126

In [83]:
f1(freedom['freedom_and_rights'], freedom['llama'])

0.833643930923096

In [86]:
mcc(freedom['freedom_and_rights'], freedom['llama'])

0.7448400510886073